In [ ]:
"""
Führt Regressionsanalyse durch

"""
import pandas as pd
import numpy as np



#Initmessage
print ("starting Classifier") 

#lade von CSV
data_frame_investigated = pd.read_csv("Data/crypto-markets.csv", sep="," )
print("data collected")

print(data_frame_investigated.head())

In [ ]:
#eliminiere redundante Attribute
REDUNDANT_ATTRIBUTES=['symbol','name'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True) #loeschen aus Dataframe
print(data_frame_investigated.head())

In [ ]:
#dt_investigated eine spalte namens trend mit 0 hinzufügen
data_frame_investigated['Trend'] = 0
#Spalte close und Trend rausschneiden
df_cl = data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin']['close'] 
df_t  = data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin']['Trend']

#vergleiche mit nachfolgern in spalte close und bearbeite trend entsrpechend
for index in df_cl.index.tolist():
    if df_cl[index] < max(df_cl.index.tolist()):
        if df_cl[index] < df_cl[index+1]:
            df_t.add(0)
        else:
            df_t.add(1)

            
print(type(df_t))
print(type(data_frame_investigated))

df_tt = pd.DataFrame(data=df_t, columns=['Trend'])
print("neu: ", type(df_tt))
print(df_tt.head())


            
#merge trend mit dt_investigated
data_frame_investigated['TMP'] = 777
df_tt['TMP'] = 777

df_result = pd.merge(data_frame_investigated, df_tt, on=['TMP'])
print(df_result)



#print(df_t)
#result = pd.concat([data_frame_investigated, df_t], axis = 1, ignore_index=True)
#print(result)
            

In [ ]:

from sklearn.model_selection import train_test_split #library fuer Split
from sklearn import linear_model #lineare Regression
import math #import fuer msqError
from sklearn.metrics import mean_squared_error

#dependency :patsy

def cryptoregress(crypto_id, feature_to_compare,crypto_id_feature_name="slug"):
    """
    lineare Regression zur Bestimmung eines Features anhand gleichen Features der anderen Cryptos
    """
    #Cryptos Gruppieren
    
    #ich kann nur anhand von Tagen schliessen, an denen die Crypto Existiert
    #erstmal alle Tage holen an denen die Crypto gehandelt wird
    dates_of_crypto_trade= data_frame_investigated[
        data_frame_investigated[crypto_id_feature_name] == crypto_id
    ]['date']
    
   
    print(dates_of_crypto_trade.head())
    
    #dann die Tupel Filtern, nur die Tage an denen Crypto getraded wird uebernehmen,
    #nur compare feature uebernehmen
    data_frame_id_exists= data_frame_investigated[
        data_frame_investigated['date'].isin(dates_of_crypto_trade)
    ][['date',crypto_id_feature_name,feature_to_compare]]
    
    #daten Pivotieren so dass Cryptos zu columns werden
    data_frame_id_exists= data_frame_id_exists.pivot(
        index='date', columns=crypto_id_feature_name, values=feature_to_compare
        ).reset_index()
    
 
    
    data_frame_id_exists=data_frame_id_exists.fillna(value=0)
    #data_frame_id_exists=data_frame_id_exists.dropna()
    
   
    #date rausloeschen
    data_frame_id_exists.drop('date',axis=1,inplace=True)
    print("cleaned Frame")
    print(data_frame_id_exists.head())
    
    ###############Train Test Split:
    #Ergebnis vom Features trennen:
    Y = data_frame_id_exists[[crypto_id]]
    data_frame_id_exists.drop(crypto_id,axis=1, inplace = True)
    print("Regress to:")
    print(Y.head())

    
        
    feature_train, feature_test, y_train, y_test = train_test_split (
        data_frame_id_exists,
        Y,
        test_size=0.1
    )

    ##############Regressionsmodell machen
    regression_model = linear_model.LinearRegression(
        fit_intercept=True, normalize=True, copy_X=True, n_jobs=-1)


        #regressionsmodell fitten
    regression_model.fit(feature_train, y_train)
    print("RSquared train:",regression_model.score(feature_train,y_train ))
    print("RSquared test:",regression_model.score(feature_test,y_test ))
    print("MeanSqure train:", math.sqrt( mean_squared_error(regression_model.predict(feature_test),y_test)))
    print("MeanSqure test:", math.sqrt( mean_squared_error(regression_model.predict(feature_train),y_train)))
    
    return regression_model 
       

    


regression_result=cryptoregress('bitcoin','close')

    
    
    

In [ ]:
#zeige mir die Koeffizienten
for cryptoiterator, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_result.coef_[0][cryptoiterator]))

In [ ]:
max(regression_result.coef_)